In [1]:
import FinanceDataReader as fdr

In [2]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import datetime
import shutil

In [3]:
def get_name(symbol, market="KRX"):
    root = "dataset/stock_list"
    
    today = datetime.datetime.now()
    today = datetime.datetime.strftime(today, "%Y_%m_%d")
    
    filename = "stock_list_" + today + ".xlsx"
    save_path = osp.join(root, filename)
    
    if not osp.isfile(save_path):
        if osp.exists(root):
            shutil.rmtree(root)
        os.mkdir(root)
        df = fdr.StockListing("KRX")
        df.set_index("Symbol", inplace=True)
        name = df.loc["060310", "Name"]
        df.to_excel(save_path)
    
    else:
        df = pd.read_excel(save_path, index_col="Symbol")
        name = df.loc[symbol, "Name"]
        
    return name

In [4]:
def get_exists_excel(symbol, start_date, end_date):
    if end_date is None:
        end_date = datetime.datetime.now()
        end_date = datetime.datetime.strftime(end_date, "%Y-%m-%d")
        
    root = "dataset"
    filename = "_".join([symbol, start_date.replace("-", "_"), end_date.replace("-", "_")]) + ".xlsx"
    save_path = osp.join(root, filename)
    
    if osp.isfile(save_path):
        is_exsists = True
        df = pd.read_excel(save_path, index_col="Date")
        
    else:
        is_exsists = False
        df = None
        
    return is_exsists, df

In [5]:
def save_excel(df, symbol, start_date, end_date):
    if end_date is None:
        end_date = datetime.datetime.now()
        end_date = datetime.datetime.strftime(end_date, "%Y-%m-%d")
        
    root = "dataset"
    if not osp.exists(root):
        os.mkdir(root)
        
    filename = "_".join([symbol, start_date.replace("-", "_"), end_date.replace("-", "_")]) + ".xlsx"
    save_path = osp.join(root, filename)
    df.to_excel(save_path)

In [6]:
def get_bollinger_band_back_testing(symbol="005930", start_date="1990-01-01", end_date=None, market="KRX"):
    is_exsists, df = get_exists_excel(symbol, start_date, end_date)
    if not is_exsists:
        df = fdr.DataReader(symbol, start=start_date, end=end_date)
        df["M20"] = df["Close"].rolling(20).mean()
        df["std"] = df["Close"].rolling(20).std()
        df["HighLine"] = df["M20"] + df["std"] * 2
        df["LowLine"] = df["M20"] - df["std"] * 2
        save_excel(df, symbol, start_date, end_date)
        
    cond = df["Close"] <= df["LowLine"]
    close_cond = df.loc[cond, "Close"]
    current_price = df["Close"].iloc[-1]
    
    buy_dict = dict()
    for date, close in close_cond.items():
        date = "_".join([str(date.year), str(date.month)]) # 2020_10 year_month
        if date not in buy_dict:
            buy_dict[date] = close
    date_list = list(buy_dict.keys())
    close_list = list(buy_dict.values())
    _start_date = start_date[:7].replace("-", "_")
    name = get_name(symbol, market)
    bought_dict = {"종목명": name, "종목코드": symbol, "보유수량": 0, "매입금액": 0}

    for idx in range(len(close_list)):
        date = date_list[idx]

        if idx == 0:
            before_date = _start_date
        else:
            before_date = date_list[idx-1]

        year = int(date.split("_")[0])
        month = int(date.split("_")[1])
        before_year = int(before_date.split("_")[0])
        before_month = int(before_date.split("_")[1])

        dif_year = year - before_year
        dif_month = month - before_month

        dif_month = dif_year * 12 + dif_month

        bought_dict["보유수량"] += dif_month
        bought_dict["매입금액"] += dif_month*close_list[idx]
    bought_dict["평균단가"] = int(bought_dict["매입금액"] / bought_dict["보유수량"])
    bought_dict["현재가"] = current_price
    bought_dict["수익률"] = round((bought_dict["현재가"]-bought_dict["평균단가"])/bought_dict["평균단가"]*100, 2)
    bought_dict["평가손익"] = int(bought_dict["매입금액"]*bought_dict["수익률"]/100)
    
    for key, value in bought_dict.items():
        if key in ["종목명", "종목코드"]:
            continue
        suffix = ""
        prefix = ""
        
        if key in ["매입금액", "평균단가", "현재가", "평가손익"]:
            suffix = "원"
        
        elif key in ["보유수량"]:
            suffix = "주"
        
        elif key in ["수익률"]:
            suffix = "%"
        
        if key in ["수익률", "평가손익"]:
            if value > 0:
                prefix = "+"
        
        value = prefix+format(value, ",")+suffix
        bought_dict[key] = value
    return bought_dict

In [7]:
get_bollinger_band_back_testing("079160", "2019-01-01")

{'종목명': 'CJ CGV',
 '종목코드': '079160',
 '보유수량': '11주',
 '매입금액': '366,414원',
 '평균단가': '33,310원',
 '현재가': '31,340원',
 '수익률': '-5.91%',
 '평가손익': '-21,655원'}